## Part 1: CUDA C++ (Primary)

### GPU Check

In [ ]:
!nvcc --version
!nvidia-smi --query-gpu=name,memory.total,memory.bandwidth.total --format=csv

### What is Memory Coalescing?

**Memory coalescing** is when the GPU combines multiple memory accesses from threads in a warp into fewer memory transactions.

```
WARP (32 threads)
┌──────────────────────────────────────────────────────────────┐
│ T0  T1  T2  T3  T4  T5  ... T31                              │
│  ↓   ↓   ↓   ↓   ↓   ↓       ↓                               │
└──────────────────────────────────────────────────────────────┘

COALESCED ACCESS (1 transaction for 128 bytes):
┌────────────────────────────────────────────────────────────────┐
│ data[0] data[1] data[2] data[3] ... data[31]                   │
└────────────────────────────────────────────────────────────────┘
  Thread 0 → data[0], Thread 1 → data[1], etc.

STRIDED ACCESS (32 separate transactions!):
┌────┐     ┌────┐     ┌────┐
│d[0]│     │d[32]│    │d[64]│  ...
└────┘     └────┘     └────┘
  T0 ↑       T1 ↑       T2 ↑
```

**Key Rule:** Consecutive threads should access consecutive memory addresses.

In [ ]:
%%writefile coalescing_demo.cu
/**
 * Memory Coalescing Demo
 * 
 * Compare coalesced vs strided memory access patterns.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define N (1 << 24)  // 16 million elements
#define BLOCK_SIZE 256
#define ITERATIONS 100

// ✅ COALESCED: Thread i accesses element i
__global__ void coalescedAccess(float *input, float *output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = input[idx] * 2.0f;
    }
}

// ❌ STRIDED: Thread i accesses element i*stride (poor coalescing)
__global__ void stridedAccess(float *input, float *output, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int strided_idx = (idx * stride) % n;
    if (idx < n) {
        output[idx] = input[strided_idx] * 2.0f;
    }
}

float benchmark(void (*kernel)(float*, float*, int), float *d_in, float *d_out, int n) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int blocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Warmup
    kernel<<<blocks, BLOCK_SIZE>>>(d_in, d_out, n);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        kernel<<<blocks, BLOCK_SIZE>>>(d_in, d_out, n);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return ms / ITERATIONS;
}

float benchmarkStrided(float *d_in, float *d_out, int n, int stride) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int blocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Warmup
    stridedAccess<<<blocks, BLOCK_SIZE>>>(d_in, d_out, n, stride);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        stridedAccess<<<blocks, BLOCK_SIZE>>>(d_in, d_out, n, stride);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return ms / ITERATIONS;
}

int main() {
    printf("=== Memory Coalescing Benchmark ===\n");
    printf("Array size: %d elements (%.2f MB)\n\n", N, N * sizeof(float) / (1024.0 * 1024.0));
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    
    // Initialize
    float *h_input = (float*)malloc(N * sizeof(float));
    for (int i = 0; i < N; i++) h_input[i] = 1.0f;
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    // Benchmark coalesced
    float coalesced_time = benchmark(coalescedAccess, d_input, d_output, N);
    float coalesced_bw = (2.0f * N * sizeof(float)) / (coalesced_time * 1e6);  // GB/s
    
    printf("✅ Coalesced Access:\n");
    printf("   Time: %.3f ms\n", coalesced_time);
    printf("   Bandwidth: %.2f GB/s\n\n", coalesced_bw);
    
    // Benchmark strided with different strides
    int strides[] = {2, 4, 8, 16, 32};
    for (int s = 0; s < 5; s++) {
        int stride = strides[s];
        float strided_time = benchmarkStrided(d_input, d_output, N, stride);
        float strided_bw = (2.0f * N * sizeof(float)) / (strided_time * 1e6);
        float slowdown = strided_time / coalesced_time;
        
        printf("❌ Stride=%d Access:\n", stride);
        printf("   Time: %.3f ms (%.1fx slower)\n", strided_time, slowdown);
        printf("   Bandwidth: %.2f GB/s\n\n", strided_bw);
    }
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 coalescing_demo.cu -o coalescing_demo && ./coalescing_demo

### Key Observations

1. **Coalesced access** achieves near-peak memory bandwidth
2. **Stride-2** roughly halves the effective bandwidth
3. **Stride-32** (warp size) is particularly bad - each thread hits a different cache line

### Array of Structures vs Structure of Arrays

A common coalescing problem:

In [ ]:
%%writefile aos_vs_soa.cu
/**
 * Array of Structures (AoS) vs Structure of Arrays (SoA)
 * 
 * SoA is almost always better for GPU memory access.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define N (1 << 22)  // 4 million particles
#define BLOCK_SIZE 256
#define ITERATIONS 100

// ❌ Array of Structures (AoS) - Poor coalescing
struct ParticleAoS {
    float x, y, z;    // Position
    float vx, vy, vz; // Velocity
};

// ✅ Structure of Arrays (SoA) - Good coalescing
struct ParticlesSoA {
    float *x, *y, *z;
    float *vx, *vy, *vz;
};

__global__ void updateAoS(ParticleAoS *particles, float dt, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Strided access: each thread reads 6 floats spread apart
        particles[idx].x += particles[idx].vx * dt;
        particles[idx].y += particles[idx].vy * dt;
        particles[idx].z += particles[idx].vz * dt;
    }
}

__global__ void updateSoA(float *x, float *y, float *z,
                          float *vx, float *vy, float *vz,
                          float dt, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Coalesced access: consecutive threads read consecutive floats
        x[idx] += vx[idx] * dt;
        y[idx] += vy[idx] * dt;
        z[idx] += vz[idx] * dt;
    }
}

int main() {
    printf("=== AoS vs SoA Benchmark ===\n");
    printf("Particles: %d\n\n", N);
    
    // Allocate AoS
    ParticleAoS *d_aos;
    cudaMalloc(&d_aos, N * sizeof(ParticleAoS));
    
    // Allocate SoA
    float *d_x, *d_y, *d_z, *d_vx, *d_vy, *d_vz;
    cudaMalloc(&d_x, N * sizeof(float));
    cudaMalloc(&d_y, N * sizeof(float));
    cudaMalloc(&d_z, N * sizeof(float));
    cudaMalloc(&d_vx, N * sizeof(float));
    cudaMalloc(&d_vy, N * sizeof(float));
    cudaMalloc(&d_vz, N * sizeof(float));
    
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    float dt = 0.01f;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark AoS
    updateAoS<<<blocks, BLOCK_SIZE>>>(d_aos, dt, N);  // Warmup
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        updateAoS<<<blocks, BLOCK_SIZE>>>(d_aos, dt, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float aos_time;
    cudaEventElapsedTime(&aos_time, start, stop);
    aos_time /= ITERATIONS;
    
    // Benchmark SoA
    updateSoA<<<blocks, BLOCK_SIZE>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, dt, N);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        updateSoA<<<blocks, BLOCK_SIZE>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, dt, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float soa_time;
    cudaEventElapsedTime(&soa_time, start, stop);
    soa_time /= ITERATIONS;
    
    printf("❌ Array of Structures (AoS): %.3f ms\n", aos_time);
    printf("✅ Structure of Arrays (SoA): %.3f ms\n", soa_time);
    printf("\nSpeedup: %.2fx\n", aos_time / soa_time);
    
    // Cleanup
    cudaFree(d_aos);
    cudaFree(d_x); cudaFree(d_y); cudaFree(d_z);
    cudaFree(d_vx); cudaFree(d_vy); cudaFree(d_vz);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 aos_vs_soa.cu -o aos_vs_soa && ./aos_vs_soa

### Matrix Transpose - A Classic Coalescing Problem

Matrix transpose is challenging because:
- Reading rows is coalesced
- Writing to columns is strided (or vice versa)

We'll solve this with shared memory in Day 2!

In [ ]:
%%writefile transpose_naive.cu
/**
 * Naive Matrix Transpose - Shows the coalescing problem
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define WIDTH 4096
#define HEIGHT 4096
#define TILE_DIM 32
#define ITERATIONS 100

// Copy kernel (coalesced read AND write) - for comparison
__global__ void copy(float *out, float *in, int width, int height) {
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    if (x < width && y < height) {
        int idx = y * width + x;
        out[idx] = in[idx];
    }
}

// Naive transpose (coalesced read, strided write)
__global__ void transposeNaive(float *out, float *in, int width, int height) {
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    if (x < width && y < height) {
        int in_idx = y * width + x;      // Row-major read (coalesced)
        int out_idx = x * height + y;    // Column-major write (strided!)
        out[out_idx] = in[in_idx];
    }
}

int main() {
    printf("=== Matrix Transpose Benchmark ===\n");
    printf("Matrix: %d x %d\n\n", WIDTH, HEIGHT);
    
    size_t size = WIDTH * HEIGHT * sizeof(float);
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    
    dim3 block(TILE_DIM, TILE_DIM);
    dim3 grid((WIDTH + TILE_DIM - 1) / TILE_DIM, (HEIGHT + TILE_DIM - 1) / TILE_DIM);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark copy
    copy<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        copy<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float copy_time;
    cudaEventElapsedTime(&copy_time, start, stop);
    copy_time /= ITERATIONS;
    float copy_bw = 2.0f * size / (copy_time * 1e6);
    
    // Benchmark naive transpose
    transposeNaive<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        transposeNaive<<<grid, block>>>(d_out, d_in, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float transpose_time;
    cudaEventElapsedTime(&transpose_time, start, stop);
    transpose_time /= ITERATIONS;
    float transpose_bw = 2.0f * size / (transpose_time * 1e6);
    
    printf("Copy (coalesced R+W):     %.3f ms, %.2f GB/s\n", copy_time, copy_bw);
    printf("Naive Transpose (strided): %.3f ms, %.2f GB/s\n", transpose_time, transpose_bw);
    printf("\nTranspose is %.2fx slower due to uncoalesced writes\n", transpose_time / copy_time);
    printf("\n💡 We'll fix this with shared memory tomorrow!\n");
    
    cudaFree(d_in);
    cudaFree(d_out);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 transpose_naive.cu -o transpose_naive && ./transpose_naive

### Memory Coalescing Summary

| Pattern | Description | Performance |
|---------|-------------|-------------|
| Coalesced | Thread i accesses data[i] | ✅ Best |
| Strided | Thread i accesses data[i*stride] | ❌ Poor |
| AoS | struct { x,y,z } particles[N] | ❌ Poor |
| SoA | float x[N], y[N], z[N] | ✅ Best |

**Key Rules:**
1. Consecutive threads should access consecutive addresses
2. Prefer Structure of Arrays (SoA) over Array of Structures (AoS)
3. When one direction must be strided, use shared memory as a staging area

---

## Part 2: Python/Numba (Optional)

The same concepts apply in Python/Numba:

In [ ]:
# Setup for Numba
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

from numba import cuda
import numpy as np
import time

print(f"CUDA Device: {cuda.get_current_device().name.decode()}")

In [ ]:
# Coalesced vs Strided in Python

@cuda.jit
def coalesced_kernel(input_arr, output_arr):
    idx = cuda.grid(1)
    if idx < input_arr.size:
        output_arr[idx] = input_arr[idx] * 2.0

@cuda.jit
def strided_kernel(input_arr, output_arr, stride):
    idx = cuda.grid(1)
    if idx < output_arr.size:
        strided_idx = (idx * stride) % input_arr.size
        output_arr[idx] = input_arr[strided_idx] * 2.0

# Test
N = 1 << 24
input_arr = cuda.to_device(np.ones(N, dtype=np.float32))
output_arr = cuda.device_array(N, dtype=np.float32)

threads = 256
blocks = (N + threads - 1) // threads

# Warmup and time coalesced
coalesced_kernel[blocks, threads](input_arr, output_arr)
cuda.synchronize()

start = time.perf_counter()
for _ in range(100):
    coalesced_kernel[blocks, threads](input_arr, output_arr)
cuda.synchronize()
coalesced_time = (time.perf_counter() - start) / 100 * 1000

print(f"✅ Coalesced: {coalesced_time:.3f} ms")

# Time strided
strided_kernel[blocks, threads](input_arr, output_arr, 32)
cuda.synchronize()

start = time.perf_counter()
for _ in range(100):
    strided_kernel[blocks, threads](input_arr, output_arr, 32)
cuda.synchronize()
strided_time = (time.perf_counter() - start) / 100 * 1000

print(f"❌ Strided (stride=32): {strided_time:.3f} ms")
print(f"\nStrided is {strided_time/coalesced_time:.1f}x slower")